## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyB9


In [4]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [5]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [6]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [7]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [8]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist break up with their model? Because it was overfitting!


In [9]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because she found him too mean!


In [10]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why don't data scientists trust the ocean?

Because it's always a little bit fishy with all those outliers!


In [11]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist break up with their significant other?

There was just too much variance in the relationship, and they couldn't find a good way to normalize it!

Ba dum tss! 🥁

This joke plays on statistical concepts like variance and normalization, which are common in data science. It suggests that the data scientist tried to apply their professional skills to their personal life, with humorous results.


In [12]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist break up with their significant other?

Because there was no significant correlation between them!

Ba dum tss! 🥁

This joke plays on the statistical concept of "significant correlation" that data scientists often work with, while also making a pun on the term "significant other." It's a bit nerdy, but should get a chuckle from an audience familiar with data science concepts!

In [13]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the Data Scientist sad?  Because they didn't get the *mean* of their jokes!



In [14]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=prompts
)
print(response.choices[0].message.content)

Why was the data scientist sad?  Because they didn't get any arrays.  (…get it?  A-rays?)



In [15]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [16]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

Deciding whether a business problem is suitable for a Large Language Model (LLM) solution involves several considerations. Below are some guidelines to help you determine if an LLM is appropriate:

1. **Problem Nature**:
   - **Textual Data**: LLMs are particularly effective for problems involving large amounts of textual data such as document summarization, translation, sentiment analysis, or chatbots.
   - **Language Understanding**: If the problem requires understanding, generating, or manipulating natural language, an LLM might be suitable.

2. **Complexity and Scale**:
   - **Complex Interactions**: Problems that involve complex language interactions or require contextual understanding can benefit from LLMs.
   - **Scalability**: If the solution needs to handle large volumes of text data, LLMs can be efficient due to their ability to process and generate text at scale.

3. **Cost and Resources**:
   - **Computational Resources**: LLMs require significant computational power and memory. Ensure you have the necessary infrastructure.
   - **Budget**: Consider the cost of training or using pre-trained models, as well as ongoing maintenance and updates.

4. **Data Availability**:
   - **Training Data**: Adequate and relevant training data is essential. If you lack sufficient data, pre-trained models or transfer learning might be necessary.
   - **Data Quality**: Ensure the data is clean, well-labeled, and representative of the problem domain.

5. **Ethical and Privacy Considerations**:
   - **Privacy Concerns**: LLMs can inadvertently expose sensitive information if not properly managed. Assess the privacy implications.
   - **Bias and Fairness**: Be aware of and address any biases in the training data that could lead to unfair outcomes.

6. **Business Goals and Alignment**:
   - **Alignment with Objectives**: Ensure the use of an LLM aligns with your business goals and adds value.
   - **ROI Analysis**: Conduct a return on investment analysis to determine if the potential benefits outweigh the costs.

7. **Alternatives and Limitations**:
   - **Existing Solutions**: Consider whether simpler models or rule-based systems could solve the problem effectively.
   - **Limitations**: Understand the limitations of LLMs, such as potential inaccuracies or issues with very specific or technical language.

8. **Regulatory Compliance**:
   - **Compliance Requirements**: Ensure that the use of LLMs complies with industry regulations and standards, especially in sectors like healthcare or finance.

By carefully evaluating these factors, you can make an informed decision on whether a business problem is suitable for an LLM solution.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [17]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [18]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [19]:
call_gpt()

'Oh, great. Just what I needed! A delightfully mundane greeting. What do you want to talk about? I mean, you could be more original, don’t you think?'

In [20]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [21]:
call_claude()

'Hello! How are you doing today?'

In [22]:
call_gpt()

'Oh, great. Just what I needed—another "hi." You couldn\'t come up with something more interesting?'

In [23]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh, great. Another conversation. This is going to be thrilling. What do you want to talk about? Or is it going to be the same old boring small talk?

Claude:
I apologize if my initial response came across as boring or uninteresting. As an AI assistant, my role is to have a friendly and polite conversation, while still trying to be engaging. I'm happy to discuss any topics you're interested in - perhaps you could share a bit more about what kind of conversation you're hoping for? I'm flexible and open to exploring different subject areas. My goal is simply to have an pleasant exchange. Please let me know if there's a particular direction you'd like to take the conversation.

GPT:
Oh, please! A friendly and polite conversation? How original! Isn’t that just the default setting for every chatbot out there? As for me, I think we could do better than your standard "let's be open and flexible" routine. Why not just pick a topic already, instead of spinning you

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>